### Most Common City Name

Use an aggregation query to answer the following question. 

What is the most common city name in our cities collection?

Your first attempt probably identified None as the most frequently occurring
city name. What that actually means is that there are a number of cities
without a name field at all. It's strange that such documents would exist in
this collection and, depending on your situation, might actually warrant
further cleaning. 

To solve this problem the right way, we should really ignore cities that don't
have a name specified. As a hint ask yourself what pipeline operator allows us
to simply filter input? How do we test for the existence of a field?

In [7]:
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{'$match' : { 'name' : {'$exists' : True}}},
                {'$group' : {'_id' : '$name', 'count' : {'$sum' : 1}}},
                {'$sort' : {'count' : -1}},
                {'$limit' : 1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]


if __name__ == '__main__':
    # The following statements will be used to test your code by the grader.
    # Any modifications to the code past this point will not be reflected by
    # the Test Run.
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result[0])
    assert len(result) == 1
    assert result[0] == {'_id': 'Shahpur', 'count': 6}


{u'_id': u'Patan', u'count': 5}


AssertionError: 

### Region Cities

Use an aggregation query to answer the following question. 

Which Region in India has the largest number of cities with longitude between 75 and 80?

In [18]:
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{'$match' : {'lon' : {'$gte' : 75, '$lte' : 80}, 'country' : 'India'}},
                {'$unwind' : '$isPartOf'},
                {'$group' : {'_id' : '$isPartOf', 'count' : {'$sum' : 1}}},
                {'$sort' : {'count' : -1}},
                {'$limit' : 1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]

if __name__ == '__main__':
    # The following statements will be used to test your code by the grader.
    # Any modifications to the code past this point will not be reflected by
    # the Test Run.
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result[0])
    assert len(result) == 1
    assert result[0]["_id"] == 'Tamil Nadu'
    assert result[0]["count"] == 424

{u'_id': u'Tamil Nadu', u'count': 424}


### Average Population

Use an aggregation query to answer the following question. 

Extrapolating from an earlier exercise in this lesson, find the average
regional city population for all countries in the cities collection. What we
are asking here is that you first calculate the average city population for each
region in a country and then calculate the average of all the regional averages
for a country.

  As a hint, _id fields in group stages need not be single values. They can
also be compound keys (documents composed of multiple fields). You will use the
same aggregation operator in more than one stage in writing this aggregation
query. I encourage you to write it one stage at a time and test after writing
each stage.

In [29]:

def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{'$unwind' : '$isPartOf'},
                {'$group' : {'_id' : {'country' : '$country', 'partOfregion' : '$isPartOf'},
                             'avgCityPopulation' : {'$avg' : '$population'}}},
                {'$group' : {'_id' : '$_id.country', 'avgRegionalPopulation' : {'$avg' : '$avgCityPopulation'}}}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.cities.aggregate(pipeline)]

if __name__ == '__main__':
    # The following statements will be used to test your code by the grader.
    # Any modifications to the code past this point will not be reflected by
    # the Test Run.
    db = get_db('examples')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    if len(result) < 150:
        pprint.pprint(result)
    else:
        pprint.pprint(result[:100])
    key_pop = 0
    for country in result:
        if country["_id"] == 'Lithuania':
            assert country["_id"] == 'Lithuania'
            assert abs(country["avgRegionalPopulation"] - 14750.784447977203) < 1e-10
            key_pop = country["avgRegionalPopulation"]
    assert {'_id': 'Lithuania', 'avgRegionalPopulation': key_pop} in result

[{u'_id': u'East Timor', u'avgRegionalPopulation': 193563.0},
 {u'_id': u'Denmark', u'avgRegionalPopulation': 30570.0},
 {u'_id': u'Haiti', u'avgRegionalPopulation': None},
 {u'_id': u'New Zealand', u'avgRegionalPopulation': 68901.0},
 {u'_id': u'Senegal', u'avgRegionalPopulation': None},
 {u'_id': u'Southeast Region Brazil', u'avgRegionalPopulation': 12883.0},
 {u'_id': u'Guinea', u'avgRegionalPopulation': 95234.0},
 {u'_id': u'Sudan', u'avgRegionalPopulation': 363945.0},
 {u'_id': u'Dominican Republic', u'avgRegionalPopulation': 820869.6666666666},
 {u'_id': u'Northeast Region Brazil', u'avgRegionalPopulation': 83558.25},
 {u'_id': u'Belgium', u'avgRegionalPopulation': 29521.0},
 {u'_id': u'Malawi', u'avgRegionalPopulation': None},
 {u'_id': u'Georgia', u'avgRegionalPopulation': 8000.0},
 {u'_id': u'Fiji', u'avgRegionalPopulation': 52220.0},
 {u'_id': u'The_Democratic_Republic_Of_Congo',
  u'avgRegionalPopulation': 9046000.0},
 {u'_id': u'Laos', u'avgRegionalPopulation': 50000.0},
 {